# General Usage
## Training start

1) Run the first cell, when its done click "RESTART RUNTIME"

2) Run the second cell

3) Run the last cell to execute


- If you go over in GPU you can restart the runtime and then do steps 2 and three above.


## Drive Mount + Git Pull Push

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
!pip install -r requirements.txt
!pip install pillow --upgrade

Mounted at /content/drive
/content/drive/Shareddrives/EECS 442 - Project/eecs442-proj
     |████████████████████████████████| 11.6MB 10.5MB/s 
     |████████████████████████████████| 2.2MB 54.0MB/s 
     |████████████████████████████████| 81kB 2.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


     |████████████████████████████████| 3.0MB 7.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 8.1.2
    Uninstalling Pillow-8.1.2:
      Successfully uninstalled Pillow-8.1.2


In [1]:
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
%load_ext autoreload
%autoreload 2

/content/drive/Shareddrives/EECS 442 - Project/eecs442-proj


In [12]:
!git config --global user.email "wzahm@umich.edu"
!git config --global user.name "Wyler Zahm"
#!git add Executor.ipynb diode_meta.json metrics.py model_pipeline.py requirements.txt train_model.py
#!git commit -m "working gpu training"
#!git push
!git status

!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Executor.ipynb
	modified:   model_pipeline.py
	modified:   train_model.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	00019_00183_indoors_000_010.png
	__pycache__/
	densenet169-b2777c0a.pth
	diode/val.tar.gz
	main.txt
	models/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
/bin/bash: vim: command not found


## Data Downloading

In [ ]:
%cd diode
!wget http://diode-dataset.s3.amazonaws.com/val.tar.gz
!tar -zxvf val.tar.gz
%cd ..

!wget https://download.pytorch.org/models/densenet169-b2777c0a.pth

Uninstalling matplotlib-3.1.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.1.3
  Using cached https://files.pythonhosted.org/packages/ce/63/74c0b6184b6b169b121bb72458818ee60a7d7c436d7b1907bd5874188c55/matplotlib-3.4.1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


## Model Exection

In [4]:
from model_pipeline import main as exec_model
from PIL import Image

Image.open('./00019_00183_indoors_000_010.png') # Verify PIL is working.
exec_model(use_cuda=True, batch_size=4)

Using GPU.
Batch size: 4


0it [00:00, ?it/s]

Model sent to GPU

Training and Evaluating Basemodel with: 	LR = 0.0001	WD = 0

Epoch #0
107


2it [00:02,  1.02s/it]


AxisError: ignored